## 1. Pre-process C file with gcc -E

In [1]:
%%sh
pwd
gcc -E -nostdinc -D'__attribute__(x)=' -Ipycparser-release_v2.17/utils/fake_libc_include -Ic-files/redis/src -Ic-files/redis/deps/lua/src c-files/redis/src/redis.c  > c-files/redis/src/redis.c.pp
echo TAIL of PRE-processes redis.c.pp
tail c-files/redis/src/redis.c.pp


/notebooks
TAIL of PRE-processes redis.c.pp

    if (server.maxmemory > 0 && server.maxmemory < 1024*1024) {
        redisLog(3,"WARNING: You specified a maxmemory value that is less than 1MB (current value is %llu bytes). Are you sure this is what you really want?", server.maxmemory);
    }

    aeSetBeforeSleepProc(server.el,beforeSleep);
    aeMain(server.el);
    aeDeleteEventLoop(server.el);
    return 0;
}


In [2]:
from pycparser import parse_file, c_ast, c_generator
from rnn_c_model import c_generator
from rnn_c_model.c_json import from_dict, to_dict, to_json, from_json

## 2. Parse preprocessed C -> Abstract Syntax Tree (AST)

In [3]:
ast = parse_file("c-files/redis/src/redis.c.pp")

## 3. Convert AST -> JSON

In [4]:
json = to_json(ast, sort_keys=True, indent=4)

#### print resulting JSON fragment

In [5]:
start = json.find("FuncCall")
print(json[start-2000:start+3000])

,
                                    "type": {
                                        "_nodetype": "TypeDecl",
                                        "coord": "c-files/redis/src/rio.h:95",
                                        "declname": "bytes_to_write",
                                        "quals": [],
                                        "type": {
                                            "_nodetype": "IdentifierType",
                                            "coord": "c-files/redis/src/rio.h:95",
                                            "names": [
                                                "size_t"
                                            ]
                                        }
                                    }
                                },
                                {
                                    "_nodetype": "If",
                                    "cond": {
                                        "_nodetype": "StructRef",
   

## 4. Generate (restore) C source from JSON

In [6]:
ast2=from_json(json)

generator = c_generator.CGenerator()
with open("c-files/redis/src/redis.c.pp.c", "w") as text_file:
    print(generator.visit(ast2), file=text_file)


### .. and verify result with cpp -E

In [7]:
%%sh
cpp -E  c-files/redis/src/redis.c.pp.c > c-files/redis/src/redis.c.pp.c.pp

#### print sampled 100 lines

In [8]:
with open("c-files/redis/src/redis.c.pp.c.pp", "r") as text_file:
    for line in text_file.readlines()[2000:2100]: print (line, end='')

int dictObjKeyCompare(void *privdata, const void *key1, const void *key2)
{
  const robj *o1 = key1;
  const robj *o2 = key2;
  return dictSdsKeyCompare(privdata, o1->ptr, o2->ptr);
}

unsigned int dictObjHash(const void *key)
{
  const robj *o = key;
  return dictGenHashFunction(o->ptr, sdslen((sds) o->ptr));
}

unsigned int dictSdsHash(const void *key)
{
  return dictGenHashFunction((unsigned char *) key, sdslen((char *) key));
}

unsigned int dictSdsCaseHash(const void *key)
{
  return dictGenCaseHashFunction((unsigned char *) key, sdslen((char *) key));
}

int dictEncObjKeyCompare(void *privdata, const void *key1, const void *key2)
{
  robj *o1 = (robj *) key1;
  robj *o2 = (robj *) key2;
  int cmp;
  if ((o1->encoding == 1) && (o2->encoding == 1))
    return o1->ptr == o2->ptr;

  o1 = getDecodedObject(o1);
  o2 = getDecodedObject(o2);
  cmp = dictSdsKeyCompare(privdata, o1->ptr, o2->ptr);
  decrRefCount(o1);
  decrRefCount(o2);
  return cmp;
}

unsigned int dictEncObjHash(const v